In [183]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
import pandas as pd

load_dotenv()
engine = create_engine(f"postgresql://{os.environ['DB_USERNAME']}:{os.environ['DB_PASSWORD']}@{os.environ['DB_HOST']}:{os.environ['DB_PORT']}/{os.environ['DB_DATABASE']}")

In [184]:
game_table = "game"
df_ = pd.read_sql_table(table_name=game_table, con=engine)

In [216]:
df = df_

In [217]:
df

,id,name,platform,rating,genre,num_of_player_idx,img_url,major_genre,split_genres,split_platform
0,1,#DRIVE,Switch,all,"Racing, Arcade, Automobile",1,https://static.metacritic.com/images/products/...,"Racing, Arcade","[Racing, Arcade]",[Switch]
1,2,#IDARB,Xbox One,all,"Action, Miscellaneous, General, Party / Minigame",5,https://static.metacritic.com/images/products/...,"Arcade, Action","[Arcade, Action]",[Xbox One]
2,3,#KILLALLZOMBIES,Xbox One,13+,"Action, Shooter, Shoot-Em-Up, Top-Down",1,https://static.metacritic.com/images/products/...,"Shooting, Action","[Shooting, Action]",[Xbox One]
3,4,'Splosion Man,Xbox 360,10+,"Action, General, Platformer, Platformer, 2D, 2D",4,https://static.metacritic.com/images/products/...,Action,[Action],[Xbox 360]
4,5,.detuned,PlayStation 3,all,"Action, Miscellaneous, Rhythm, Music, Music",1,https://static.metacritic.com/images/products/...,"Arcade, Action","[Arcade, Action]",[PlayStation 3]
...,...,...,...,...,...,...,...,...,...,...
12899,12900,rain,PlayStation 3,10+,"Miscellaneous, Action Adventure, Fantasy, Gene...",1,https://static.metacritic.com/images/products/...,"Action, Arcade, Adventure","[Action, Arcade, Adventure]",[PlayStation 3]
12900,12901,theHunter: Call of the Wild,PC,13+,"Sports, Individual, Nature, Hunting",5,https://static.metacritic.com/images/products/...,"Sports, Shooting","[Sports, Shooting]",[PC]
12901,12902,uDraw Studio,Wii,all,"Miscellaneous, General, General",5,https://static.metacritic.com/images/products/...,None,[None],[Wii]
12902,12903,void tRrLM(); //Void Terrarium,Switch,13+,"Role-Playing, Roguelike",1,https://static.metacritic.com/images/products/...,Arcade,[Arcade],[Switch]


In [218]:
#example
age = 20             # 9 10 13 17 20
platform = ["PC", "PlayStation 4"]
players = 1
major_genre = ["Tactics", "Puzzle"]
tag = ["Graphics", "Completion", "easy"]
games = ["Zombie Driver: Immortal Edition", "Zumba Fitness Rush"]

In [219]:
def age_filter(df, age):
    if age == 9:
        return df[df['rating'] == 'all']
    elif age == 10:
        return df[(df['rating'] == 'all') | (df['rating'] == '10+')]
    elif age == 13:
        return df[(df['rating'] == 'all') | (df['rating'] == '10+') | (df['rating'] == '13+')]
    elif age == 17:
        return df[(df['rating'] == 'all') | (df['rating'] == '10+') | (df['rating'] == '13+') | (df['rating'] == '17+')]
    else: # age == 20:
        return df
    
    
def players_filter(df, players):
    return df[df['num_of_player_idx'] == players]


def platform_and_genre_filter(df_, platform, major_genre):
    df = df_.copy()
    df['split_genres'] = df['major_genre'].apply(lambda genres: genres.split(','))
    df['split_platform'] = df['platform'].apply(lambda genres: genres.split(','))

    idx_arr = []
    for genre, platforms, idx in zip(df['split_genres'],df['split_platform'], df['id']):
        i_ = [x.strip() for x in genre]
        j_ = [y.strip() for y in platform]

        if any(x in i_ for x in major_genre) and any(y in j_ for y in platform): idx_arr.append(idx-1)

    # 'split_genres' 열을 삭제
    df = df.drop('split_genres', axis=1)
    df = df.drop('split_platform', axis=1)
    return df.loc[idx_arr]
    

def major_genre_filter(df_, major_genre):
    # major_genre 열에서 각 장르를 개별적으로 추출하여 새로운 열인 'split_genres'에 저장
    df = df_.copy()
    df['split_genres'] = df['major_genre'].apply(lambda genres: genres.split(','))
    
    idx_arr = []
    for genre, idx in zip(df['split_genres'], df['id']):
        i_ = [x.strip() for x in genre]
        if any(x in i_ for x in major_genre): idx_arr.append(idx-1)

    # 'split_genres' 열을 삭제
    #df = df.drop('split_genres', axis=1)
    
    return df_.loc[idx_arr]
    

def filter (df, age, platform, players, major_genre):
    output = platform_and_genre_filter(df, platform, major_genre)
    output = players_filter(output, players)
    output = age_filter(output, age)

    return output

In [220]:
# 필터링
df = filter(df, age, platform, players, major_genre)

In [221]:
df

,id,name,platform,rating,genre,num_of_player_idx,img_url,major_genre
20,21,0RBITALIS,PC,17+,"Miscellaneous, Puzzle, Action",1,https://static.metacritic.com/images/products/...,"Puzzle, Action"
32,33,112 Operator,PC,17+,"Strategy, General",1,https://static.metacritic.com/images/products/...,Tactics
41,42,1950s Lawn Mower Kids,DS,all,"Miscellaneous, Puzzle, Action, General, General",1,https://static.metacritic.com/images/products/...,"Puzzle, Action"
42,43,1954: Alcatraz,PC,17+,"General, Puzzle, Miscellaneous, Adventure, Puz...",1,https://static.metacritic.com/images/products/...,"Puzzle, Adventure"
58,59,39 Days to Mars,PC,17+,"Puzzle, Action",1,https://static.metacritic.com/images/products/...,"Puzzle, Action"
...,...,...,...,...,...,...,...,...
12876,12877,echochrome ii,PlayStation 3,all,"Miscellaneous, Puzzle, Action, Puzzle, General...",1,https://static.metacritic.com/images/products/...,"Puzzle, Action"
12884,12885,iO,"PlayStation 4, Xbox One",all,"Puzzle, Action",1,https://static.metacritic.com/images/products/...,"Puzzle, Action"
12885,12886,ilomilo,Xbox 360,all,"Miscellaneous, Puzzle, Action, General, Puzzle...",1,https://static.metacritic.com/images/products/...,"Puzzle, Action"
12886,12887,imprint-X,PC,17+,"Puzzle, Action",1,https://static.metacritic.com/images/products/...,"Puzzle, Action"


In [222]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [224]:
user_df = pd.DataFrame(columns=['id', 'name', 'genre'])
for i in games:
    tmp = df_[df_['name'] == i]
    user_df = pd.concat([user_df, tmp[['id', 'name', 'genre']]], ignore_index=True)

user_df

,id,name,genre
0,12833,Zombie Driver: Immortal Edition,"Action, Simulation, General, Vehicle, Combat"
1,12864,Zumba Fitness Rush,"Miscellaneous, Rhythm, Exercise / Fitness, Dan..."


In [225]:
db_df = df[['name', 'genre']]

In [226]:
db_df.head()

,name,genre
20,0RBITALIS,"Miscellaneous, Puzzle, Action"
32,112 Operator,"Strategy, General"
41,1950s Lawn Mower Kids,"Miscellaneous, Puzzle, Action, General, General"
42,1954: Alcatraz,"General, Puzzle, Miscellaneous, Adventure, Puz..."
58,39 Days to Mars,"Puzzle, Action"


In [228]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 기존 게임 데이터프레임과 유저 게임 데이터프레임 통합
combined_df = pd.concat([db_df, user_df], ignore_index=True)

# TF-IDF 벡터화 객체 생성
vectorizer = TfidfVectorizer()

# 게임 장르 데이터를 TF-IDF 벡터로 변환
genre_vectors = vectorizer.fit_transform(combined_df['genre'])

# 추천을 위한 유사도 측정
similarities = cosine_similarity(genre_vectors[-2:], genre_vectors[:-2])
top_similar_indices = similarities.argsort()[0][::-1][:5] #5개 추천

# 추천 게임 목록 생성
recommendations = combined_df.loc[top_similar_indices, 'name'].tolist()

# 결과 출력
print(f"\n추천하는 게임:")
for i, game in enumerate(recommendations):
    print(f"{i+1}. {game}")


추천하는 게임:
1. Zombie Driver: Immortal Edition
2. Bugs vs. Tanks!
3. Daemon X Machina
4. Euro Truck Simulator 2
5. SOL: Exodus
